Repo being followed: https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

1. Add Data to Remo
    - Dataset format {x_min, y_min, w, h} -> {x_min, y_min, x_max, y_max} for Remo.
    - Export annotation from Remo after visualizing. ((Test if both match)
    - Cannot retain image height, image width in Plain CSV (Potential Problem) (While exporting, needed at pre-processsing time)
2. Pre-processing for EfficientDet or YOLO
    - Boxes need to be normalized before passing them as input: -_-
        - Formula: {((x + (w / 2)) / im_w), ((y + (h / 2) / im_h),  (w / im_w),  (h / im_h)}
        - **w**: Bounding Box width (x_max - x_min)
        - **h**: Bounding Box height (y_max - y_min)
        - **im_h**: image height
        - **im_w**: image width
        - **x**: x_min
        - **y**: y_min
    - Output of this formula:
        - {center_x, center_y, bbox_w, bbox_h}
    - Desirable format of feeding the data to the model:
        img_name_path, class, center_x, center_y, bbox_w, bbox_h
    - Current models then build a im_name.txt file for EVERY image (Source: [YOLOv5](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data))
    - Then there is directory structure to be followed for the same. (This is dataset agnostic)
    - Create a CSV of text file paths? Will refer Kaggle Kernels for more info, this is longer amount of work.
    -  
3. Model:
    - Model options can be YOLO, EfficientDet, and if everything works out in Step #2, I doubt it will be difficult.
4. Inference:
    - For now, I assume it is straightforward and carries over from the Model repo.
5. Visualization:
    - Post Process boxes to format file_name, classes, x_min, y_min, x_max, y_max in the form of a Plain CSV.

**Work In Progress**

Might just completely switch to mmdetection format.

In [29]:
### Imports
import os
import cv2
import csv
import glob
import pandas as pd
import numpy as np


In [15]:
### Pre-Pre-Processing FOR YOLOv5
train = pd.read_csv("./non_image_files/CSVs/train.csv")

train['bbox'] = train['bbox'].apply(lambda x: x[1:-1].split(","))
train['x_min'] = train['bbox'].apply(lambda x: x[0]).astype('float32')
train['y_min'] = train['bbox'].apply(lambda x: x[1]).astype('float32')
train["classes"] = 0
train["x_max"] = train.apply(lambda col: col.x_min + float(col.bbox[2]), axis=1)
train["y_max"] = train.apply(lambda col: col.y_min + float(col.bbox[3]), axis = 1)
train = train[['image_id', "classes", 'x_min', 'y_min', "x_max", "y_max"]]

train.rename(columns={"image_id" : "file_name"}, inplace=True)

""" This will then be added to Remo via remo.create_datast() """


' This will then be added to Remo via remo.create_datast() '

In [21]:
subset = train[0:100]
subset.head()

,file_name,classes,x_min,y_min,x_max,y_max
0,b6ab77fd7,0,834.0,222.0,890.0,258.0
1,b6ab77fd7,0,226.0,548.0,356.0,606.0
2,b6ab77fd7,0,377.0,504.0,451.0,664.0
3,b6ab77fd7,0,834.0,95.0,943.0,202.0
4,b6ab77fd7,0,26.0,144.0,150.0,261.0


In [22]:
img_h, img_w = 1024, 1024
input_df= pd.DataFrame(columns=["file_name", "classes", "x", "y", "w", "h"])
input_df["file_name"] = subset["file_name"]
input_df["classes"] = subset["classes"]
input_df["w"] = subset.apply(lambda col: (col.x_max - col.x_min) / img_w , axis=1)
input_df["h"] = subset.apply(lambda col: (col.y_max - col.y_min) / img_h, axis=1)
input_df["x"] = subset.apply(lambda col: (col.x_min + (col.x_max - col.x_min) / 2) / img_w, axis=1)
input_df["y"] = subset.apply(lambda col: (col.y_min + (col.x_max - col.x_min) / 2) / img_h, axis=1)




In [23]:
input_df["file_name"] = subset["file_name"]
input_df["classes"] = subset["classes"]

![example image txt](https://user-images.githubusercontent.com/26833433/78174482-307bb800-740e-11ea-8b09-840693671042.png)

This is the expected format for one image in YOLOv5

,file_name,classes,x,y,w,h
0,b6ab77fd7,0,0.841797,0.244141,0.054688,0.035156
1,b6ab77fd7,0,0.284180,0.598633,0.126953,0.056641
2,b6ab77fd7,0,0.404297,0.528320,0.072266,0.156250
3,b6ab77fd7,0,0.867676,0.145996,0.106445,0.104492
4,b6ab77fd7,0,0.085938,0.201172,0.121094,0.114258
...,...,...,...,...,...,...
95,7b72ea0fb,0,0.154785,0.484863,0.063477,0.051758
96,7b72ea0fb,0,0.387695,0.289062,0.066406,0.141602
97,7b72ea0fb,0,0.937988,0.687988,0.094727,0.074219
98,7b72ea0fb,0,0.222656,0.139648,0.089844,0.082031
